Exercise from Think Stats, 2nd Edition (thinkstats2.com)<br>
Allen Downey

In [19]:
%matplotlib inline

Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool. 

In [20]:
import nsfg
import statsmodels.formula.api as smf
import numpy as np

df = nsfg.ReadFemPreg()

In [21]:
#Cleaning data
#converting all babies value (not ascertained (value 7), don't know (value 9)) and female (2)) 
#to 0 except male babies (value 1)
df.loc[df['babysex'] != 1, 'babysex'] = 0 

baby = df[df.outcome == 1 & (df.prglngth >= 30)]

In [22]:
"""
def r2 (column_no):
    try:
        variable = str(baby.columns[column_no])
        formula ='prglngth ~' + variable
        model = smf.ols(formula, data = baby)
        result = model.fit()
        return (result.rsquared, result.f_pvalue)
    except: 
        return (0,0)


variable_r2 = np.empty(len(baby.columns), dtype=object)
variable_p = np.empty(len(baby.columns), dtype=object)

for i in range(0, len(baby.columns)-1):
    variable_r2[i], variable_p[i] = r2(i)
    print baby.columns[i], variable_r2[i], variable_p[i]
"""

"\ndef r2 (column_no):\n    try:\n        variable = str(baby.columns[column_no])\n        formula ='prglngth ~' + variable\n        model = smf.ols(formula, data = baby)\n        result = model.fit()\n        return (result.rsquared, result.f_pvalue)\n    except: \n        return (0,0)\n\n\nvariable_r2 = np.empty(len(baby.columns), dtype=object)\nvariable_p = np.empty(len(baby.columns), dtype=object)\n\nfor i in range(0, len(baby.columns)-1):\n    variable_r2[i], variable_p[i] = r2(i)\n    print baby.columns[i], variable_r2[i], variable_p[i]\n"

In [23]:
import chap01soln
resp = chap01soln.ReadFemResp()
resp.index = resp.caseid
join = baby.join(resp, on='caseid', rsuffix='_r')
    
t = []
for name in join.columns:
    try:
        if join[name].var() < 1e-7:
            continue

        formula = ('prglngth ~' + name).encode('ascii')
        model = smf.ols(formula, data=join)
        if model.nobs < len(join)/2:
            continue

        results = model.fit()
        
        if name == 'babysex':
            print results.rsquared
            
    except (ValueError, TypeError):
        continue

    t.append((results.rsquared, name))

0.000193556647438


In [24]:
t.sort(reverse=True)
for mse, name in t[:30]:
    print name, mse

prglngth 1.0
wksgest 0.84834444419
totalwgt_lb 0.179548090415
lbw1 0.172861427756
birthwgt_lb 0.171898841283
mosgest 0.121330423471
prglngth_i 0.0152701066371
nbrnaliv 0.00769581202714
canhaver 0.00593856647669
datcon01_i 0.00430171280399
pregend1 0.00391214836732
con1mar1_i 0.00389093115987
anynurse 0.0033435395396
bfeedwks 0.00331167840872
rmarout08_i 0.00267785152974
marout08_i 0.00267785152974
coh1dur 0.00263088703117
datend02_i 0.00251119531636
datcon02_i 0.00251119531636
bpa_bdscheck1 0.00249843519185
gestasun_w 0.0024438427747
everhyst 0.00233317151081
totincr 0.00230344114738
agecon02_i 0.00230286994584
hyst 0.00229607320666
wantbold 0.00227411957336
ageprg02_i 0.002245700018
p1yrage 0.00220709648186
mar1con1_i 0.00219110817676
marcon08_i 0.00218870780021


Sorting out columns which gave r^2 > 0.0025,

In [25]:
sig_variable = []
unknown_list = ['prglngth', 'wksgest', 'totalwgt_lb', 'birthwgt_lb', 'lbw1, mosgest', 
                'prglngth_i', 'canhaver', 'lbw1', 'mosgest', 'nbrnaliv', 'anynurse',
                'bfeedwks', 'pregend1', 'rmarout11_i', 'datend02_i', 'marout11_i']
for mse, name in t[:30]:
    if (mse > 0.0025) & (name not in unknown_list):
        sig_variable.append ((mse, name))


for mse, name in sig_variable[:30]:
    print name, mse

datcon01_i 0.00430171280399
con1mar1_i 0.00389093115987
rmarout08_i 0.00267785152974
marout08_i 0.00267785152974
coh1dur 0.00263088703117
datcon02_i 0.00251119531636


datcon01_i: DATCON01 (CM date when pregnancy began - 1st ) Imputation Flag  

mar1con1_i: MAR1CON1 (Months btw/1st marriage & 1st conception (or interview)) Imputation Flag

marcon11_i: MARCON11 (Formal marital status when pregnancy began - 11th ) Imputation Flag 

datcon02_i: DATCON02 (CM date when pregnancy began - 2nd) Imputation Flag

cmlastlb: CM for R's most recent live birth

Removing variables with less significant r^2 values and information the co-workers cannot know beforehand, the variables we are left with are mainly imputation flags which are not good indicators to base the prediction on. Also, they are not variables which the co-worker might know.
Guessing might be a better option.

## Clarifying Questions

Use this space to ask questions regarding the content covered in the reading. These questions should be restricted to helping you better understand the material. For questions that push beyond what is in the reading, use the next answer field. If you don't have a fully formed question, but are generally having a difficult time with a topic, you can indicate that here as well.

## Enrichment Questions

Use this space to ask any questions that go beyond (but are related to) the material presented in this reading. Perhaps there is a particular topic you'd like to see covered in more depth. Perhaps you'd like to know how to use a library in a way that wasn't show in the reading. One way to think about this is what additional topics would you want covered in the next class (or addressed in a followup e-mail to the class). I'm a little fuzzy on what stuff will likely go here, so we'll see how things evolve.

I would like a more through discussion on regression.

## Additional Resources / Explorations

If you found any useful resources, or tried some useful exercises that you'd like to report please do so here. Let us know what you did, what you learned, and how others can replicate it.